In [7]:
import json
# Read the JSON data from the file
with open("../course_full_v2.json") as json_file:
    course_data = json.load(json_file)

In [13]:
course_data.items()

dict_items([('1', [{'Module_heading': '1. Introduction to Indian History and Culture', 'subtopic_output': '1. Ancient Indian Civilization', 'content_output': 'Ancient Indian Civilization:\n\n• The ancient Indian civilization emerged around 2500 BCE in the Indus Valley, which is now modern-day Pakistan and northwest India.\n   - Example: Excavations at Harappa and Mohenjo-Daro reveal well-planned cities with advanced infrastructure like roads, drainage systems, and public baths.\n\n• The civilization was known for its advanced urban planning and efficient agricultural practices.\n   - Example: The cities had a grid-like layout with houses made of baked bricks, and the agricultural fields were irrigated using sophisticated canal systems.\n\n• The people of ancient India were skilled in various crafts and industries.\n   - Example: Archaeological evidence shows that they were skilled in pottery, metallurgy, jewelry making, and cotton weaving.\n\n• Religion played a significant role in the

In [15]:
# Iterate through the nested dictionaries
for module_number, subtopics in course_data.items():
    for subtopic_data in subtopics:
        module_heading = subtopic_data["Module_heading"]
        subtopic_output = subtopic_data["subtopic_output"]
        content_output = subtopic_data["content_output"]
        
        print(f"Module Number: {module_number}")
        print(f"Module Heading: {module_heading}")
        print(f"Subtopic: {subtopic_output}")
        print(f"Content: {content_output}")
        print("\n")

Module Number: 1
Module Heading: 1. Introduction to Indian History and Culture
Subtopic: 1. Ancient Indian Civilization
Content: Ancient Indian Civilization:

• The ancient Indian civilization emerged around 2500 BCE in the Indus Valley, which is now modern-day Pakistan and northwest India.
   - Example: Excavations at Harappa and Mohenjo-Daro reveal well-planned cities with advanced infrastructure like roads, drainage systems, and public baths.

• The civilization was known for its advanced urban planning and efficient agricultural practices.
   - Example: The cities had a grid-like layout with houses made of baked bricks, and the agricultural fields were irrigated using sophisticated canal systems.

• The people of ancient India were skilled in various crafts and industries.
   - Example: Archaeological evidence shows that they were skilled in pottery, metallurgy, jewelry making, and cotton weaving.

• Religion played a significant role in the lives of ancient Indians.
   - Example: 